# Test for game with observabilities

In [28]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [29]:
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.parsers as parsers
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner

import source.util as util
import numpy as np
import pandas as pd
from importlib import *
import source.util as util
from copy import deepcopy, copy

In [30]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(runner)
reload(parsers)
reload(game)
reload(environment)
reload(util)

<module 'source.util' from 'C:\\Users\\Mattia\\Desktop\\Mattia\\Git\\onlineSG\\source\\util.py'>

In [15]:
time_horizon = 1000

In [16]:
T = 5 #np.random.randint(2,10)
targets = util.gen_targets(T, low=1, high=10)
distributions = []
for i in range(20):
    distributions.append(tuple(util.gen_distr(T)))
values = tuple((v, v) for v in targets)
print(values)
observabilities = util.gen_observabilities_with_len(T)
g = game.GameWithObservabilities(values, time_horizon)
print(T, targets)
print(observabilities)
g.set_observabilities(observabilities)
attacker = [attackers.ObservingStackelbergAttacker(g, 1, 1)]
profiles = []
profiles.append(attackers.ObservingStackelbergAttacker(g, 1, 1))
for d in distributions:
    profiles.append(attackers.StochasticAttacker(g, 1, 1, *d))
profiles.append(attackers.StackelbergAttacker(g, 1))
defender = [base_defenders.StackelbergDefender(g, 0, 1)]
#g.set_players(defender, attacker, profiles)

((3, 3), (2, 2), (1, 1), (8, 8), (4, 4))
5 [3, 2, 1, 8, 4]
{0: 0.686, 1: 0.756, 2: 0.823, 3: 0.918, 4: 0.806}


In [28]:
e = runner.Experiment(g)

In [29]:
e.run_interaction_with_observations()
print(e.game.strategy_history[-1])
print(e.agent.avg_rewards)
print(util.game_str(e.game, lenght=7))

GurobiError: No Gurobi license found (user mattia, host LAPTOP-FK21MAJ8, hostid 901c63d0, cores 2)

In [31]:
m = util.gen_tar_with_len(7)

In [32]:
print(m)

[0.983, 0.721, 0.997, 0.869, 0.761, 0.721, 0.877]


In [36]:
mval = tuple((v, v) for v in m)
#g1 = game.GameWithObservabilities(mval, time_horizon)
M = [0, 1, 2, 3, 4, 5, 6,]
#g1.set_players(defender, attacker, profiles)
print (mval)

((0.983, 0.983), (0.721, 0.721), (0.997, 0.997), (0.869, 0.869), (0.761, 0.761), (0.721, 0.721), (0.877, 0.877))


In [39]:
import numpy as np
import re
from copy import deepcopy
import enum
from source.errors import AlreadyFinalizedError
from math import log
#print(g1.values[1][1])
#k=[g1.values[0][1] * int(i != t) for i in M]
print (k)
A_ub = []
for t in M:
    terms = [mval[t][1] * int(i != t) for i in M]
    terms += [1]
    A_ub.append(terms)
b_ub = [0 for i in range(len(A_ub))]
A_eq = [[1 for i in M] + [0]]
b_eq = [1]
bounds = [(0, 1) for i in M] + [(None, None)]

[0.865, 0.865, 0.865, 0.865, 0.865, 0.865, 0.865, 0.0]


In [40]:
print(A_ub, b_ub, A_eq, b_eq)

[[0.0, 0.983, 0.983, 0.983, 0.983, 0.983, 0.983, 1], [0.721, 0.0, 0.721, 0.721, 0.721, 0.721, 0.721, 1], [0.997, 0.997, 0.0, 0.997, 0.997, 0.997, 0.997, 1], [0.869, 0.869, 0.869, 0.0, 0.869, 0.869, 0.869, 1], [0.761, 0.761, 0.761, 0.761, 0.0, 0.761, 0.761, 1], [0.721, 0.721, 0.721, 0.721, 0.721, 0.0, 0.721, 1], [0.877, 0.877, 0.877, 0.877, 0.877, 0.877, 0.0, 1]] [0, 0, 0, 0, 0, 0, 0] [[1, 1, 1, 1, 1, 1, 1, 0]] [1]
